# Domain adaptive Pretraining
#### Language: `Arabic`

Continual pretraining on the unlablled translated corpus to improve the performance of the model.

In [16]:
import sys
sys.path.append('../')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import warnings # to ignore warnings
from utils import squad_json_to_dataframe_train
warnings.filterwarnings('ignore')

In [21]:
# training data
language = 'vi'
input_file_path = f'../SQuAD/translate-train/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [22]:
train.head()

,index,question,context,answer_start,text,c_id
0,5726e1755951b619008f8141,Tên của nhà vật lý người Hà Lan đã phát minh r...,"Vào tháng 10 năm 1745, Ewald Georg von Kleist ...",549,Pieter van Musschenbroek,0
1,572fde9204bcaa1900d76e05,Môn thể thao Olympic nào mà Hrant Shahinyan th...,"Trước năm 1992, người Armenia sẽ tham gia Thế ...",430,thể dục dụng cụ,1
2,56d37d7f59d6e4140014651a,Vòng loại nào Chris Medina đã bị loại trong mù...,Một trong những thí sinh nổi bật hơn trong năm...,191,Top 40,2
3,57318dc8a5e9cc1400cdc061,Spielberg đã ký bao nhiêu bộ phim truyền hình?,"Dựa trên sức mạnh công việc của mình, Universa...",77,bốn,3
4,57331b0dd058e614000b56fd,Lý thuyết chủ yếu của thực tế mà Whitehead phả...,Bắt đầu từ cuối những năm 1910 và đầu những nă...,469,thực tế được xây dựng cơ bản bởi các bit vật c...,4


In [23]:
train.drop(["index","answer_start","text","c_id"],axis = 1,inplace=True)
train.head()

,question,context
0,Tên của nhà vật lý người Hà Lan đã phát minh r...,"Vào tháng 10 năm 1745, Ewald Georg von Kleist ..."
1,Môn thể thao Olympic nào mà Hrant Shahinyan th...,"Trước năm 1992, người Armenia sẽ tham gia Thế ..."
2,Vòng loại nào Chris Medina đã bị loại trong mù...,Một trong những thí sinh nổi bật hơn trong năm...
3,Spielberg đã ký bao nhiêu bộ phim truyền hình?,"Dựa trên sức mạnh công việc của mình, Universa..."
4,Lý thuyết chủ yếu của thực tế mà Whitehead phả...,Bắt đầu từ cuối những năm 1910 và đầu những nă...


In [20]:
pretraining_data_path = f"./squad.pretraining.{language}.csv"
train.to_csv(pretraining_data_path,index=False)

In [6]:
from transformers import AutoTokenizer, LineByLineTextDataset

# model_name = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=pretraining_data_path, block_size=128)

In [13]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, AutoModelForMaskedLM

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True, mlm_probability=0.15)    

 
training_args = TrainingArguments(
    output_dir="Ar-Mulitlingula-MiniLM",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=8,
    save_total_limit=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
model_name = "microsoft/Multilingual-MiniLM-L12-H384"
model = AutoModelForMaskedLM.from_pretrained(model_name)

loading configuration file https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384/resolve/main/config.json from cache at C:\Users\Subha/.cache\huggingface\transformers\12a5ad52cb7fc5542e16e354fe6eb487f2f87edac63bf85dc238b1236dbaf24c.ccf88548169a21266c411bcf65585ba761d762a9c85fde572f529806fdd94ee2
Model config BertConfig {
  "_name_or_path": "microsoft/Multilingual-MiniLM-L12-H384",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

loading weights file https://huggingface.co/microso

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset)

trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 86788
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10849


RuntimeError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 6.00 GiB total capacity; 4.39 GiB already allocated; 0 bytes free; 4.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF